<a href="https://colab.research.google.com/github/Manya-65/assigment1/blob/main/MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('ToyotaCorolla - MLR.csv')
df.head()

,Price,Age_08_04,KM,Fuel_Type,HP,Automatic,cc,Doors,Cylinders,Gears,Weight
0,13500,23,46986,Diesel,90,0,2000,3,4,5,1165
1,13750,23,72937,Diesel,90,0,2000,3,4,5,1165
2,13950,24,41711,Diesel,90,0,2000,3,4,5,1165
3,14950,26,48000,Diesel,90,0,2000,3,4,5,1165
4,13750,30,38500,Diesel,90,0,2000,3,4,5,1170


**Reasoning**:
Create dummy variables for 'Fuel_Type', concatenate them with the original DataFrame, drop the original 'Fuel_Type' column, check for missing values again, and consider outlier handling for numerical columns based on previous EDA.



In [13]:
# Outlier Handling (using IQR method)
numerical_cols = df_processed.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Exclude the target variable 'Price' from outlier handling
numerical_cols.remove('Price')

for col in numerical_cols:
    Q1 = df_processed[col].quantile(0.25)
    Q3 = df_processed[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap the outliers
    df_processed[col] = df_processed[col].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

print("\nDataFrame after outlier handling:")
display(df_processed.describe())


DataFrame after outlier handling:


,Price,Age_08_04,KM,HP,Automatic,cc,Doors,Cylinders,Gears,Weight
count,1436.000000,1436.000000,1436.000000,1436.00000,1436.0,1436.000000,1436.000000,1436.0,1436.0,1436.000000
mean,10730.824513,55.958914,67604.873172,101.10376,0.0,1558.565460,4.033426,4.0,5.0,1068.579387
std,3626.964585,18.566512,34738.595454,13.14745,0.0,168.970222,0.952677,0.0,0.0,37.774625
min,4350.000000,5.000000,1.000000,69.00000,0.0,1300.000000,2.000000,4.0,5.0,1000.000000
25%,8450.000000,44.000000,43000.000000,90.00000,0.0,1400.000000,3.000000,4.0,5.0,1040.000000
50%,9900.000000,61.000000,63389.500000,110.00000,0.0,1600.000000,4.000000,4.0,5.0,1070.000000
75%,11950.000000,70.000000,87020.750000,110.00000,0.0,1600.000000,5.000000,4.0,5.0,1085.000000
max,32500.000000,80.000000,153051.875000,140.00000,0.0,1900.000000,5.000000,4.0,5.0,1152.500000


In [15]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df_processed.drop('Price', axis=1)
y = df_processed['Price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape (X_train, y_train):", X_train.shape, y_train.shape)
print("Testing set shape (X_test, y_test):", X_test.shape, y_test.shape)

Training set shape (X_train, y_train): (1148, 11) (1148,)
Testing set shape (X_test, y_test): (288, 11) (288,)


In [16]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Model 1: Using selected features based on initial EDA and domain knowledge
selected_features_1 = ['Age_08_04', 'KM', 'HP', 'Automatic', 'Weight', 'Fuel_Type_Petrol']
X_train_m1 = X_train[selected_features_1]
X_test_m1 = X_test[selected_features_1]

model_1 = LinearRegression()
model_1.fit(X_train_m1, y_train)

print("Model 1 (Selected Features) Coefficients:")
for feature, coef in zip(selected_features_1, model_1.coef_):
    print(f"{feature}: {coef}")

# Model 2: Using more features
selected_features_2 = ['Age_08_04', 'KM', 'HP', 'Automatic', 'cc', 'Doors', 'Weight', 'Fuel_Type_Petrol']
X_train_m2 = X_train[selected_features_2]
X_test_m2 = X_test[selected_features_2]

model_2 = LinearRegression()
model_2.fit(X_train_m2, y_train)

print("\nModel 2 (More Features) Coefficients:")
for feature, coef in zip(selected_features_2, model_2.coef_):
    print(f"{feature}: {coef}")

# Model 3: Using all available features
X_train_m3 = X_train
X_test_m3 = X_test

model_3 = LinearRegression()
model_3.fit(X_train_m3, y_train)

print("\nModel 3 (All Features) Coefficients:")
for feature, coef in zip(X_train.columns, model_3.coef_):
    print(f"{feature}: {coef}")

Model 1 (Selected Features) Coefficients:
Age_08_04: -124.11959764743983
KM: -0.02060304450809105
HP: 21.81691206766012
Automatic: -2.5579538487363607e-13
Weight: 25.09139202701839
Fuel_Type_Petrol: 627.2560996839766

Model 2 (More Features) Coefficients:
Age_08_04: -119.94702454256942
KM: -0.019641504046222127
HP: 40.72091948618096
Automatic: 8.171241461241152e-14
cc: -2.9615431613889553
Doors: -140.58403118218277
Weight: 31.878910466130428
Fuel_Type_Petrol: -331.22434449289676

Model 3 (All Features) Coefficients:
Age_08_04: -116.09059707059085
KM: -0.01724055891762677
HP: 120.52550903875488
Automatic: -1.4921397450962104e-13
cc: -9.68233201758307
Doors: -119.77366969074535
Cylinders: 0.0
Gears: 9.094947017729282e-13
Weight: 31.444409966724166
Fuel_Type_Diesel: 6603.666715581272
Fuel_Type_Petrol: 1532.210420079217


In [17]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Evaluate Model 1
y_pred_m1 = model_1.predict(X_test_m1)
r2_1 = r2_score(y_test, y_pred_m1)
mae_1 = mean_absolute_error(y_test, y_pred_m1)
mse_1 = mean_squared_error(y_test, y_pred_m1)
rmse_1 = np.sqrt(mse_1)

print("Model 1 Evaluation:")
print(f"R-squared: {r2_1}")
print(f"MAE: {mae_1}")
print(f"MSE: {mse_1}")
print(f"RMSE: {rmse_1}")

# Evaluate Model 2
y_pred_m2 = model_2.predict(X_test_m2)
r2_2 = r2_score(y_test, y_pred_m2)
mae_2 = mean_absolute_error(y_test, y_pred_m2)
mse_2 = mean_squared_error(y_test, y_pred_m2)
rmse_2 = np.sqrt(mse_2)

print("\nModel 2 Evaluation:")
print(f"R-squared: {r2_2}")
print(f"MAE: {mae_2}")
print(f"MSE: {mse_2}")
print(f"RMSE: {rmse_2}")

# Evaluate Model 3
y_pred_m3 = model_3.predict(X_test_m3)
r2_3 = r2_score(y_test, y_pred_m3)
mae_3 = mean_absolute_error(y_test, y_pred_m3)
mse_3 = mean_squared_error(y_test, y_pred_m3)
rmse_3 = np.sqrt(mse_3)

print("\nModel 3 Evaluation:")
print(f"R-squared: {r2_3}")
print(f"MAE: {mae_3}")
print(f"MSE: {mse_3}")
print(f"RMSE: {rmse_3}")

Model 1 Evaluation:
R-squared: 0.8262298353173543
MAE: 1049.2143352355608
MSE: 2318578.6146958247
RMSE: 1522.687957099492

Model 2 Evaluation:
R-squared: 0.8351425585838099
MAE: 1016.8432076320767
MSE: 2199658.0301291593
RMSE: 1483.1244149191123

Model 3 Evaluation:
R-squared: 0.862517028136921
MAE: 977.8454761525949
MSE: 1834406.2631736556
RMSE: 1354.4025484226083


**Reasoning**:
Apply Lasso and Ridge regression to the model and evaluate their performance.

In [18]:
from sklearn.linear_model import Lasso, Ridge

# Apply Lasso Regression
lasso_model = Lasso(alpha=1.0) # You can tune the alpha parameter
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)

# Evaluate Lasso Model
r2_lasso = r2_score(y_test, y_pred_lasso)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)

print("Lasso Model Evaluation:")
print(f"R-squared: {r2_lasso}")
print(f"MAE: {mae_lasso}")
print(f"MSE: {mse_lasso}")
print(f"RMSE: {rmse_lasso}")

# Apply Ridge Regression
ridge_model = Ridge(alpha=1.0) # You can tune the alpha parameter
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

# Evaluate Ridge Model
r2_ridge = r2_score(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)

print("\nRidge Model Evaluation:")
print(f"R-squared: {r2_ridge}")
print(f"MAE: {mae_ridge}")
print(f"MSE: {mse_ridge}")
print(f"RMSE: {rmse_ridge}")

Lasso Model Evaluation:
R-squared: 0.8618058597557307
MAE: 978.8435977572567
MSE: 1843895.2327161953
RMSE: 1357.901039367816

Ridge Model Evaluation:
R-squared: 0.8588737840049123
MAE: 983.5005990308904
MSE: 1883017.3003331025
RMSE: 1372.2307751734409


**Reasoning**:
Summarize the performance of all models (Linear Regression with different feature sets, Lasso, and Ridge) and compare their evaluation metrics to identify the best performing model.

In [19]:
import pandas as pd

performance_summary = {
    'Model': ['Model 1 (Selected Features)', 'Model 2 (More Features)', 'Model 3 (All Features)', 'Lasso (alpha=1.0)', 'Ridge (alpha=1.0)'],
    'R-squared': [r2_1, r2_2, r2_3, r2_lasso, r2_ridge],
    'MAE': [mae_1, mae_2, mae_3, mae_lasso, mae_ridge],
    'MSE': [mse_1, mse_2, mse_3, mse_lasso, mse_ridge],
    'RMSE': [rmse_1, rmse_2, rmse_3, rmse_lasso, rmse_ridge]
}

performance_df = pd.DataFrame(performance_summary)

print("Model Performance Summary:")
display(performance_df)

# Identify the best model based on R-squared (higher is better)
best_r2_model = performance_df.loc[performance_df['R-squared'].idxmax()]
print("\nBest Model based on R-squared:")
display(best_r2_model)

# Identify the best model based on MAE (lower is better)
lowest_mae_model = performance_df.loc[performance_df['MAE'].idxmin()]
print("\nBest Model based on MAE:")
display(lowest_mae_model)

# Identify the best model based on MSE (lower is better)
lowest_mse_model = performance_df.loc[performance_df['MSE'].idxmin()]
print("\nBest Model based on MSE:")
display(lowest_mse_model)

# Identify the best model based on RMSE (lower is better)
lowest_rmse_model = performance_df.loc[performance_df['RMSE'].idxmin()]
print("\nBest Model based on RMSE:")
display(lowest_rmse_model)

Model Performance Summary:


,Model,R-squared,MAE,MSE,RMSE
0,Model 1 (Selected Features),0.826230,1049.214335,2.318579e+06,1522.687957
1,Model 2 (More Features),0.835143,1016.843208,2.199658e+06,1483.124415
2,Model 3 (All Features),0.862517,977.845476,1.834406e+06,1354.402548
3,Lasso (alpha=1.0),0.861806,978.843598,1.843895e+06,1357.901039
4,Ridge (alpha=1.0),0.858874,983.500599,1.883017e+06,1372.230775



Best Model based on R-squared:


,2
Model,Model 3 (All Features)
R-squared,0.862517
MAE,977.845476
MSE,1834406.263174
RMSE,1354.402548



Best Model based on MAE:


,2
Model,Model 3 (All Features)
R-squared,0.862517
MAE,977.845476
MSE,1834406.263174
RMSE,1354.402548



Best Model based on MSE:


,2
Model,Model 3 (All Features)
R-squared,0.862517
MAE,977.845476
MSE,1834406.263174
RMSE,1354.402548



Best Model based on RMSE:


,2
Model,Model 3 (All Features)
R-squared,0.862517
MAE,977.845476
MSE,1834406.263174
RMSE,1354.402548


**Reasoning**:
Interpret the coefficients of the Lasso and Ridge models and compare them with the coefficients of the best performing linear regression model (Model 3) to understand the impact of regularization on feature importance.

In [20]:
# Get coefficients for Model 3 (All Features)
model_3_coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Model_3_Coefficient': model_3.coef_
})

print("Model 3 (All Features) Coefficients:")
display(model_3_coefficients)

# Get coefficients for Lasso Model
lasso_coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Lasso_Coefficient': lasso_model.coef_
})

print("\nLasso Model Coefficients:")
display(lasso_coefficients)

# Get coefficients for Ridge Model
ridge_coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Ridge_Coefficient': ridge_model.coef_
})

print("\nRidge Model Coefficients:")
display(ridge_coefficients)

# Compare coefficients (optional, for deeper analysis)
# You can merge the dataframes to compare side-by-side
# combined_coefficients = model_3_coefficients.merge(lasso_coefficients, on='Feature').merge(ridge_coefficients, on='Feature')
# print("\nComparison of Coefficients:")
# display(combined_coefficients)

Model 3 (All Features) Coefficients:


,Feature,Model_3_Coefficient
0,Age_08_04,-1.160906e+02
1,KM,-1.724056e-02
2,HP,1.205255e+02
3,Automatic,-1.492140e-13
4,cc,-9.682332e+00
5,Doors,-1.197737e+02
6,Cylinders,0.000000e+00
7,Gears,9.094947e-13
8,Weight,3.144441e+01
9,Fuel_Type_Diesel,6.603667e+03



Lasso Model Coefficients:


,Feature,Lasso_Coefficient
0,Age_08_04,-116.208945
1,KM,-0.017423
2,HP,117.460609
3,Automatic,0.000000
4,cc,-9.419657
5,Doors,-118.265011
6,Cylinders,0.000000
7,Gears,0.000000
8,Weight,31.357632
9,Fuel_Type_Diesel,6286.835853



Ridge Model Coefficients:


,Feature,Ridge_Coefficient
0,Age_08_04,-116.731512
1,KM,-0.017746
2,HP,106.565306
3,Automatic,0.000000
4,cc,-8.503650
5,Doors,-122.402668
6,Cylinders,0.000000
7,Gears,0.000000
8,Weight,31.442927
9,Fuel_Type_Diesel,5384.573889
